In [27]:
import numpy as np
import pandas as pd
import math

In [7]:
case = pd.read_csv("data/data/covid_19.csv")
hos1 = pd.read_csv("data/data/hospital1.csv")
hos2 = pd.read_csv("data/data/hospital2.csv")

In [14]:
case["submission_date"] = case["submission_date"].astype("datetime64")
case = case.loc[:, ["submission_date", "state", "tot_cases", "new_case"]]
case["lag_date"] = case["submission_date"].shift(1)

In [31]:
join = pd.merge(case, case, how="left", left_on = ["submission_date", "state"], right_on = ["lag_date", "state"])
join["inc_rate"] = join["new_case_y"] / join["tot_cases_x"] 
join = join.fillna(0)

In [40]:
case_ = join.loc[np.logical_and(join["inc_rate"]!=0, join["inc_rate"] != math.inf), ["submission_date_x", "state", "inc_rate"]].reset_index(drop=True)
case_

,submission_date_x,state,inc_rate
0,2020-03-05,CO,1.000000
1,2020-03-06,CO,3.000000
2,2020-03-08,CO,0.500000
3,2020-03-09,CO,0.416667
4,2020-03-10,CO,1.000000
...,...,...,...
11785,2020-10-03,MP,0.027397
11786,2020-10-08,MP,0.026667
11787,2020-10-14,MP,0.038961
11788,2020-10-15,MP,0.075000


In [64]:
def helper(state):
    tmp = join[join["state"] == state].reset_index(drop=True)
    inf_ind = np.where(tmp.inc_rate==math.inf)[0]
    return tmp.loc[inf_ind[0]+1:, ["submission_date_x", "state", "inc_rate"]] if inf_ind else tmp.loc[:, ["submission_date_x", "state", "inc_rate"]]
    
case_ = pd.concat([helper(x) for x in np.unique(join["state"])],axis=0).reset_index(drop=True)

C:\Users\cnyin\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  after removing the cwd from sys.path.
C:\Users\cnyin\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  after removing the cwd from sys.path.
C:\Users\cnyin\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  after removing the cwd from sys.path.
C:\Users\cnyin\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The truth value of an empty

In [67]:
case_["inc_rate"]

0        0.000000
1        0.000000
2        0.000000
3        2.000000
4        1.000000
           ...   
14172    0.023091
14173    0.033802
14174    0.027518
14175    0.042099
14176    0.000000
Name: inc_rate, Length: 14177, dtype: float64